In [1]:
import os
import numpy as np
from tqdm import trange
from scipy.stats import bootstrap
rng = np.random.default_rng()

path='/mnt/disk2/marcnol/Gdrive/papers/2025_Bardou/data/flybrain_trace_files/raw_matrices'
file='combined_not_OK107_KC_ABp_rep1_Matrix_PWDscMatrix.npy'
uniquebarcodes='combined_all_traces_KC_G_Matrix_uniqueBarcodes.ecsv'

sc_matrix=np.load(path+os.sep+file)
print(f"$ scmatrix shape: {sc_matrix.shape}")

uniqueBarcodes = list(np.loadtxt(path+os.sep+uniquebarcodes, delimiter=" "))
uniqueBarcodes = [int(x) for x in uniqueBarcodes]
print(f"$ unique barcodes loaded: {uniqueBarcodes}")


$ scmatrix shape: (25, 25, 37250)
$ unique barcodes loaded: [1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 27, 28, 639, 708]


In [2]:
def bootstrapping(x,N_bootstrap=9999):
    data = (x,)  # samples must be in a sequence
    
    res = bootstrap(data, np.mean, confidence_level=0.9,
                    n_resamples=N_bootstrap,
                    batch = None,
                    random_state=rng)
    
    return res.bootstrap_distribution


def bootstraps_matrix(m,N_bootstrap=9999):
    n_bins = m.shape[0]
    mean_bs = np.zeros((n_bins,n_bins))
    mean_error = np.zeros((n_bins,n_bins))
    print(f"$ n bins: {n_bins}")    
    
    for i in trange(n_bins):
        for j in range(i+1,n_bins):
            if i != j:
                # gets distribution and removes nans
                x = m[i,j,:]
                x = x[~np.isnan(x)]
                
                # bootstraps distribution
                bs = bootstrapping(x,N_bootstrap=N_bootstrap)
 
                # gets mean and std of mean
                mean_bs[i,j] = np.median(bs)
                mean_error[i,j] = np.std(bs)
                
                # symmetrizes matrix
                mean_bs[j,i] = mean_bs[i,j]
                mean_error[j,i] = np.std(bs)

    for i in range(n_bins):
        mean_bs[i,i], mean_error[i,i] = np.nan, np.nan
                
    return mean_bs, mean_error

N_bootstrap=100
mean_bs, mean_error = bootstraps_matrix(
    sc_matrix, N_bootstrap=N_bootstrap
)
print(f"$ mean_bs = {mean_bs.shape}")


$ n bins: 25


100%|██████████| 25/25 [00:02<00:00, 10.64it/s]

$ mean_bs = (25, 25)
